- ### Import Libraries and APIs

In [1]:
# import the required libraries and APIs
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

c:\Users\soyvi\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- ### Downloading the Tensorflow `imdb_review` dataset

In [2]:
## load the imdb reviews datset
data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

- ### Segregating training and testing sets

In [3]:
## segregate training and test set
train_data, test_data = data['train'], data['test']

# create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [4]:
# iterate over the train data to extract sentences and labels
for sent, label in train_data:
    train_sentences.append(str(sent.numpy().decode('utf8')))
    train_labels.append(label.numpy())

# iterate over the test set to extract sentences and labels
for sent, label in test_data:
    test_sentences.append(str(sent.numpy().decode('utf8')))
    test_labels.append(label.numpy())

In [5]:
# convert lists to numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

- ### Data preparation `&` setting up the tokenizer

In [6]:
# define the parameters for the tokenizing and padding

vocab_size = 10000
embedding_dim = 16
max_length = 150
trunc_type = "post"
oov_tok = "<oov>"

In [7]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen=max_length, truncating=trunc_type)

# testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs, maxlen=max_length, truncating=trunc_type)

In [8]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])


print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   11   26   75  571
    6  805 2354  313  106   19   12    7  629  686    6    4 2219    5
  181  584   64 1454  110 2263    3 3951   21    2    1    3  258   41
 4677    4  174  188   21  


<h3 style="color:orange; font-weight:bold" > - Define the Neural Network with Embedding layer </h3>

1. Use the Sequential API
2. Add an embedding input layer of input size equal to vocabulary size.
3. Add a flatten layer, and two dense layers.

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile the model with loss function optimizer and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           160000    
                                                                 
 flatten_1 (Flatten)         (None, 2400)              0         
                                                                 
 dense_2 (Dense)             (None, 6)                 14406     
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 174413 (681.30 KB)
Trainable params: 174413 (681.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
first_layer_weights = model.layers[0].get_weights()[0]
print(first_layer_weights)

[[ 0.03420449 -0.00760012  0.0475404  ...  0.03849551  0.00735551
   0.04970194]
 [ 0.00293024 -0.02019421 -0.01189305 ... -0.02738918 -0.04886878
  -0.01985217]
 [ 0.04417821  0.03829149 -0.03400198 ...  0.01464446  0.03809166
   0.00796164]
 ...
 [-0.04529114  0.03091918  0.04217918 ...  0.04098041  0.02455306
   0.02421195]
 [ 0.03861166  0.01928907 -0.02539095 ...  0.03343615  0.00397048
  -0.01606653]
 [ 0.02232195  0.01020239  0.02450914 ... -0.02841574  0.0334674
  -0.03789992]]


<h3 style="color:orange; font-weight:bold" > - Model Training </h3>

In [21]:
num_epochs = 10
# training the model with training and vilidation set
model.fit(
    train_padded,
    train_labels,
    epochs = num_epochs,
    validation_data = (test_padded, test_labels)

)

Epoch 1/10



782/782 [==============================] - 11s 11ms/step - loss: 0.4716 - accuracy: 0.7544 - val_loss: 0.3577 - val_accuracy: 0.8450
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.2234 - accuracy: 0.9149 - val_loss: 0.3939 - val_accuracy: 0.8303
Epoch 3/10
782/782 [==============================] - 11s 14ms/step - loss: 0.0865 - accuracy: 0.9783 - val_loss: 0.4790 - val_accuracy: 0.8234
Epoch 4/10
782/782 [==============================] - 11s 15ms/step - loss: 0.0233 - accuracy: 0.9966 - val_loss: 0.5630 - val_accuracy: 0.8216
Epoch 5/10
782/782 [==============================] - 11s 14ms/step - loss: 0.0052 - accuracy: 0.9997 - val_loss: 0.6508 - val_accuracy: 0.8214
Epoch 6/10
782/782 [==============================] - 12s 15ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6829 - val_accuracy: 0.8259
Epoch 7/10
782/782 [==============================] - 10s 12ms/step - loss: 8.1335e-04 - accuracy: 1.0000 - val_loss: 0.7329 - val_accuracy: 0.8258

<h3 style="color:orange; font-weight:bold" > - Deriving weights from the embedding layer </h3>

In [23]:
# isolating the first embedding layer
l1 = model.layers[0]

# extract learned weights
weights = l1.get_weights()[0]
print(weights.shape)
print(weights)

(10000, 16)
[[ 1.79723985e-02 -9.03354213e-03  2.34014709e-02 ...  1.87363438e-02
   1.60090160e-02  9.78506505e-02]
 [ 3.90011147e-02 -2.70659942e-02 -3.55541259e-02 ... -7.54991546e-02
   1.34516815e-02  1.17816284e-01]
 [ 4.01189476e-02 -5.44104958e-03 -5.31555936e-02 ...  6.73783338e-03
   4.73823659e-02  1.27314672e-01]
 ...
 [-1.23997763e-01 -1.24556664e-02  1.39235288e-01 ...  1.61552966e-01
   6.30636737e-02 -5.33563793e-02]
 [ 4.59713191e-02 -7.47568980e-02  1.39058709e-01 ...  1.72270581e-01
   8.01814795e-02 -4.26091701e-02]
 [-7.77987912e-02 -1.46989487e-02  1.46190122e-01 ... -1.71850348e-04
  -1.21196680e-01  6.06914498e-02]]
